<a href="https://colab.research.google.com/github/gitesei/EnsembleLab/blob/main/EnsembleLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preliminary information:**

This Colab notebook enables running molecular dynamics (MD) simulations of intrinsically disordered proteins (IDPs) and protein regions (IDRs) and to study their conformational ensembles [1].

MD simulations employ the coarse-grained force field CALVADOS 2 [2], where each residue is mapped onto a single bead and modeled with a stickiness parameter and electrostatics.

Simulations are run using openMM [3] and only require that the user provides the sequence of an IDP, set environmental conditions (temperature & ionic strength), and the charge states of His residues and terminal amino and carboxyl groups.

Coarse-grained trajectories are converted to all-atom trajectories using the _powerful chain restoration algorithm_ (PULCHRA) [4].

Small angle X-ray scattering curves are calculated using _polynomial expansions of protein structures and interactions_ (Pepsi)-SAXS [5].

Bayesian/Maximum Entropy Reweighting is performed using the BME Python library [6].

FRET efficiencies are estimated using FRETpredict [7].

### Usage

MD simulations run on GPU. To enable GPU select `Runtime` from the menu, then `Change runtime type` and select `GPU`.

__Note:__ Cells for preliminary operations should be executed one by one to prevent crashes. This notebook uses condacolab, whose installation will cause a kernel restart. Because of this, a crash will happen during preliminary operations if you execute all cells at once.

### References

If you use this notebook, you may consider citing the following papers:

1. G. Tesei, A. I. Trolle, N. Jonsson, J. Betz, F. Pesce, K. E. Johansson, K. Lindorff-Larsen __Conformational ensembles of the human intrinsically disordered proteome: Bridging chain compaction with function and sequence conservation__ _bioRxiv_ 2023 2023.05.08.539815 DOI: https://doi.org/10.1101/2023.05.08.539815

2. G. Tesei and K. Lindorff-Larsen __Improved predictions of phase behaviour of intrinsically disordered proteins by tuning the interaction range [version 2; peer review: 2 approved]__ _Open Research Europe_ 2023 2(94) DOI: https://doi.org/10.12688/openreseurope.14967.2

3. P. Eastman, J. Swails, J. D. Chodera et al. __OpenMM 7: Rapid development of high performance algorithms for molecular dynamics__ _PLoS Computational Biology_ 2017 13(7):e1005659 DOI: https://doi.org/10.1371/journal.pcbi.1005659

4. P. Rotkiewicz and J. Skolnick __Fast procedure for reconstruction of full-atom protein models from reduced representations__ _J. Comput. Chem._ 2008 29:1460–1465 DOI: https://doi.org/10.1002/jcc.20906

5. S. Grudinin, M. Garkavenko, and A. Kazennov __Pepsi-SAXS: an adaptive method for rapid and accurate computation of small-angle x-ray scattering profiles__ _Acta Crystallogr. D Struct. Biol_ 2017 73:449–464 DOI: https://doi.org/10.1107/S2059798317005745

6. S. Bottaro, T. Bengtsen, and K. Lindorff-Larsen __Integrating Molecular Simulation and Experimental Data: A Bayesian/Maximum Entropy Reweighting Approach__ _Methods Mol. Biol._ 2020 2112:219–240 DOI: https://doi.org/10.1007/978-1-0716-0270-6_15


7. D. Montepietra, G. Tesei, J. M. Martins, M. B. A. Kunze, R. B. Best, and K. Lindorff-Larsen __FRETpredict: A Python package for FRET efficiency predictions using rotamer libraries__ bioRxiv 2023 2023.01.27.525885 DOI: https://doi.org/10.1101/2023.01.27.525885

---

Authors: Francesco Pesce and Giulio Tesei

In [ ]:
#@title <b>Preliminary operations</b>: setting the environment (i)
import subprocess
subprocess.run('pip install -q condacolab'.split())
import condacolab
condacolab.install()
import subprocess

In [ ]:
#@title <b><font color='#E3B505'>0 - Input protein sequence</font></b>
from google.colab import files
import numpy as np
import os
import shutil

try:
    os.rmdir('sample_data')
except:
    pass

#@markdown Insert here the sequence of the IDR that you want to simulate:
#NAME = "IBB" #@param {type:"string"}
#SEQUENCE = "GCTNENANTPAARLHRFKNKGKDSTEMRRRRIEVNVELRKAKKDDQMLKRRNVSSFPDDATSPLQENRNNQGTVNWSVDDIVKGINSSNVENQLQATCA" #@param {type:"string"}

#NAME = "NLS" #@param {type:"string"}
#SEQUENCE = "ACETNKRKREQISTDNEAKMQIQEEKSPKKKRKKRSSKANKPPECA" #@param {type:"string"}

NAME = "Sic1" #@param {type:"string"}
SEQUENCE = "GSMTPSTPPRSRGTRYLAQPSGNTSSSALMQGQKTPQKPSQNLVPVTPSTTKSFKNAPLLAPPNSNMGMTSPFNGLTSPQRSPFPKSSVKRT" #@param {type:"string"}

#@markdown Simulation settings:
Temperature = 296 #@param {type:"number"}
Ionic_strength = 0.150 #@param {type:"number"}
#@markdown <i>*Units: Temperature [K], Ionic_strength [M]<i>

#@markdown Define charge states:
charged_N_terminal_amine = True #@param {type:"boolean"}
charged_C_terminal_carboxyl = True #@param {type:"boolean"}
charged_histidine = True #@param {type:"boolean"}
Nc = 1 if charged_N_terminal_amine == True else 0
Cc = 1 if charged_C_terminal_carboxyl == True else 0

if charged_histidine == True:
    print('Define pH and pKa to set the charge of Histidines according to the Henderson-Hasselbalch equation.')
    pH = input('Enter pH value: ')
    pH = float(pH)
    pKa = input('Enter pKa value: ')
    pKa = float(pKa)
    Hc = 1/(1+10**(pH-pKa))
if charged_histidine == False:
    Hc = 0

np.savetxt('env_settings.txt', np.array([Temperature, Ionic_strength, Hc, Nc, Cc]).T, header='temperature ionic_strength, His_charge, N_term_charge, C_term_charge')

# Experimental data

# DOI: 10.1073/pnas.1704692114
# DOI: 10.1021/jacs.0c02088
exp_data = {'IBB': {'exp_rg':3.2, 'exp_rg_err':0.2,
                   'exp_E':.5, 'exp_E_err':0.02, 'sites':[98, 2],
                   'alexa':['488 C1R','594 C1R']},
            'NLS': {'exp_rg':2.4, 'exp_rg_err':0.3,
                   'exp_E':.79, 'exp_E_err':0.02, 'sites':[45, 2],
                   'alexa':['488 C1R','594 C1R']},
            'Sic1': {'exp_rg':2.8, 'exp_rg_err':0.2,
                   'exp_E':.42, 'exp_E_err':0.02, 'sites':[1, 90],
                   'alexa':['488 C1R','647 C2R']}}

# Need to store metadata prior to condacolab restarting the kernel
f = open('seq.fasta','w')
f.write('>{:s}\n{:s}'.format(NAME,SEQUENCE))
f.close()
try:
    os.mkdir('{:s}'.format('SAXS'))
except:
    pass
try:
    os.mkdir('{:s}'.format('FRET'))
except:
    pass

In [ ]:
#@title <b>Preliminary operations</b>: installing libraries (ii)
subprocess.run('mamba install openmm=7.7 -c conda-forge --yes'.split())
subprocess.run('pip install wget statsmodels localcider==0.1.18 fretpredict==0.1.8 MDAnalysis==2.0.0 kneed==0.5.0 matplotlib mdtraj'.split())
subprocess.run('pip uninstall scikit-learn -y'.split())
subprocess.run('pip install scikit-learn==1.0.2'.split())
import wget
wget.download('https://raw.githubusercontent.com/fpesceKU/BLOCKING/main/block_tools.py')
wget.download('https://raw.githubusercontent.com/fpesceKU/BLOCKING/main/main.py')

In [ ]:
#@title <b>Preliminary operations</b>: setting the environment (iii)
import os
import shutil
import itertools
import numpy as np
import pandas as pd
import scipy.stats as scs
from scipy.optimize import curve_fit
from sklearn.covariance import LedoitWolf
from sklearn.linear_model import LinearRegression
from scipy import constants
from numpy import linalg
from localcider.sequenceParameters import SequenceParameters
from main import BlockAnalysis
import mdtraj as md
from simtk import openmm, unit
from simtk.openmm import app
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from google.colab import files
import matplotlib_inline.backend_inline
from joblib import load
import MDAnalysis
from kneed import KneeLocator
from FRETpredict import FRETpredict
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')
from ipywidgets import interactive
import ipywidgets as widgets
from statsmodels.stats.weightstats import DescrStatsW

In [ ]:
#@title <b>Preliminary operations</b>: downloading PULCHRA, Pepsi-SAXS, BLOCKING, and BME (iv)
%%bash

rm -r sample_data

wget https://github.com/fpesceKU/EnsembleLab/raw/main/utils/pulchra &> /dev/null
chmod +x pulchra

wget https://files.inria.fr/NanoDFiles/Website/Software/Pepsi-SAXS/Linux/3.0/Pepsi-SAXS-Linux.zip &> /dev/null
unzip Pepsi-SAXS-Linux.zip &> /dev/null
rm Pepsi-SAXS-Linux.zip

wget https://raw.githubusercontent.com/fpesceKU/BLOCKING/main/block_tools.py &> /dev/null
wget https://raw.githubusercontent.com/fpesceKU/BLOCKING/main/main.py &> /dev/null

wget https://raw.githubusercontent.com/KULL-Centre/BME/main/BME_tools.py &> /dev/null
wget https://raw.githubusercontent.com/KULL-Centre/BME/main/BME.py &> /dev/null

wget https://raw.githubusercontent.com/ehb54/GenApp-BayesApp/main/bin/source/bift.f &> /dev/null
gfortran bift.f -march=native -O2 -o bift

In [ ]:
#@title <b><font color='#A79AB2'>Loading SAXS data</font></b>
print('SAXS data must be in a file containing 3 columns, which are q, I and sigma. Commented lines (#) are allowed.')
saxs_file = f'{NAME:s}.dat'
url = 'https://github.com/gitesei/EnsembleLab/blob/main/'

if os.path.exists(saxs_file) == False:
    wget.download(url+'example_data/'+saxs_file+'?raw=true')

#check data
try:
    np.loadtxt(saxs_file)
except:
    print("Unable to read file. Make sure the file exists and contains 3 columns (q, I, sigma).")
assert np.shape(np.loadtxt(saxs_file))[1] == 3, "Expected file with 3 columns (q, I, sigma)"

exp_saxs = np.loadtxt(saxs_file)
if exp_saxs[...,0][-1] < 1:
    print('q is in Å units. Converting to nm.')
    exp_saxs[...,0] = exp_saxs[...,0]*10
    np.savetxt(saxs_file, exp_saxs)

if (exp_saxs[...,0] >= 5).sum() > 0:
    print("""Found {} q-values above 5 nm^(-1).
          SAXS calculations are not reliable in that region of the spectrum.
          Those datapoints will be removed""".format((exp_saxs[...,0] >= 5).sum()))
    exp_saxs = exp_saxs[(exp_saxs[...,0] < 5)]
    np.savetxt(saxs_file, exp_saxs)

shutil.copy(saxs_file, 'saxs_input.dat')

In [ ]:
#@title <b><font color='#A79AB2'>MD Toolbox</font></b>
url = 'https://github.com/KULL-Centre/_2023_Tesei_IDRome/blob/main'

if os.path.exists('svr_model_nu.joblib') == False:
    wget.download(url+'/svr_models/svr_model_nu.joblib?raw=true')
if os.path.exists('svr_model_SPR.joblib') == False:
    wget.download(url+'/svr_models/svr_model_SPR.joblib?raw=true')
if os.path.exists('residues.csv') == False:
    wget.download(url+'/md_simulations/data/residues.csv?raw=true')

residues = pd.read_csv('residues.csv')
residues = residues.set_index('one')

def genParamsLJ(df,seq,Nc,Cc):
    fasta = seq.copy()
    r = df.copy()
    if Nc == 1:
        r.loc['X'] = r.loc[fasta[0]]
        r.loc['X','MW'] += 2
        fasta[0] = 'X'
    if Cc == 1:
        r.loc['Z'] = r.loc[fasta[-1]]
        r.loc['Z','MW'] += 16
        fasta[-1] = 'Z'
    types = list(np.unique(fasta))
    lj_eps = 0.2*4.184
    lj_sigma = pd.DataFrame((r.sigmas.values+r.sigmas.values.reshape(-1,1))/2,
                            index=r.sigmas.index,columns=r.sigmas.index)
    lj_lambda = pd.DataFrame((r.lambdas.values+r.lambdas.values.reshape(-1,1))/2,
                             index=r.lambdas.index,columns=r.lambdas.index)
    return lj_eps, lj_sigma, lj_lambda, fasta, types

def genParamsDH(df,seq,temp,ionic,Nc,Cc,Hc):
    kT = 8.3145*temp*1e-3
    fasta = seq.copy()
    r = df.copy()
    # Set the charge on HIS based on the pH of the protein solution
    r.q = r.q.astype(float)
    r.loc['H','q'] = Hc
    if Nc == 1:
        r.loc['X'] = r.loc[fasta[0]]
        r.loc['X','q'] = r.loc[seq[0],'q'] + 1.
        fasta[0] = 'X'
    if Cc == 1:
        r.loc['Z'] = r.loc[fasta[-1]]
        r.loc['Z','q'] = r.loc[seq[-1],'q'] - 1.
        fasta[-1] = 'Z'
    # Calculate the prefactor for the Yukawa potential
    fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
    epsw = fepsw(temp)
    lB = 1.6021766**2/(4*np.pi*8.854188*epsw)*6.022*1000/kT
    yukawa_eps = [r.loc[a].q*np.sqrt(lB*kT) for a in fasta]
    # Calculate the inverse of the Debye length
    yukawa_kappa = np.sqrt(8*np.pi*lB*ionic*6.022/10)
    return yukawa_eps, yukawa_kappa

def genXTC(name, eqsteps=10):
    """
    Generates coordinate and trajectory
    in convenient formats
    """
    traj = md.load("{:s}/pretraj.dcd".format(name), top="{:s}/top.pdb".format(name))
    traj = traj.image_molecules(inplace=False, anchor_molecules=[set(traj.top.chain(0).atoms)], make_whole=True)
    traj.center_coordinates()
    traj.xyz += traj.unitcell_lengths[0,0]/2
    cgtop = md.Topology()
    cgchain = cgtop.add_chain()
    for atom in traj.top.atoms:
        cgres = cgtop.add_residue(atom.name, cgchain)
        cgtop.add_atom('CA', element=md.element.carbon, residue=cgres)
    traj = md.Trajectory(traj.xyz, cgtop, traj.time, traj.unitcell_lengths, traj.unitcell_angles)
    traj[int(eqsteps):].save_xtc("{:s}/traj.xtc".format(name))
    traj[int(eqsteps)].save_pdb("{:s}/top.pdb".format(name))

def simulate(residues,name,seq,temp,ionic,Nc,Cc,Hc,nsteps,stride=1e3,eqsteps=1000):
    os.mkdir(name)

    lj_eps, _, _, fasta, types= genParamsLJ(residues,seq,Nc,Cc)
    yukawa_eps, yukawa_kappa = genParamsDH(residues,seq,temp,ionic,Nc,Cc,Hc)

    N = len(fasta)
    L = (N-1)*0.38+4

    system = openmm.System()

    # set box vectors
    a = unit.Quantity(np.zeros([3]), unit.nanometers)
    a[0] = L * unit.nanometers
    b = unit.Quantity(np.zeros([3]), unit.nanometers)
    b[1] = L * unit.nanometers
    c = unit.Quantity(np.zeros([3]), unit.nanometers)
    c[2] = L * unit.nanometers
    system.setDefaultPeriodicBoxVectors(a, b, c)

    top = md.Topology()
    pos = []
    chain = top.add_chain()
    pos.append([[0,0,L/2+(i-N/2.)*.38] for i in range(N)])
    for resname in fasta:
        residue = top.add_residue(resname, chain)
        top.add_atom(resname, element=md.element.carbon, residue=residue)
    for i in range(chain.n_atoms-1):
        top.add_bond(chain.atom(i),chain.atom(i+1))
    md.Trajectory(np.array(pos).reshape(N,3), top, 0, [L,L,L], [90,90,90]).save_pdb('{:s}/top.pdb'.format(name))

    pdb = app.pdbfile.PDBFile('{:s}/top.pdb'.format(name))

    system.addParticle((residues.loc[seq[0]].MW+2)*unit.amu)
    for a in seq[1:-1]:
        system.addParticle(residues.loc[a].MW*unit.amu)
    system.addParticle((residues.loc[seq[-1]].MW+16)*unit.amu)

    hb = openmm.openmm.HarmonicBondForce()
    energy_expression = 'select(step(r-2^(1/6)*s),4*eps*l*((s/r)^12-(s/r)^6-shift),4*eps*((s/r)^12-(s/r)^6-l*shift)+eps*(1-l))'
    ah = openmm.openmm.CustomNonbondedForce(energy_expression+'; s=0.5*(s1+s2); l=0.5*(l1+l2); shift=(0.5*(s1+s2)/2)^12-(0.5*(s1+s2)/2)^6')
    yu = openmm.openmm.CustomNonbondedForce('q*(exp(-kappa*r)/r - exp(-kappa*4)/4); q=q1*q2')
    yu.addGlobalParameter('kappa',yukawa_kappa/unit.nanometer)
    yu.addPerParticleParameter('q')

    ah.addGlobalParameter('eps',lj_eps*unit.kilojoules_per_mole)
    ah.addPerParticleParameter('s')
    ah.addPerParticleParameter('l')

    for a,e in zip(seq,yukawa_eps):
        yu.addParticle([e*unit.nanometer*unit.kilojoules_per_mole])
        ah.addParticle([residues.loc[a].sigmas*unit.nanometer, residues.loc[a].lambdas*unit.dimensionless])

    for i in range(N-1):
        hb.addBond(i, i+1, 0.38*unit.nanometer, 8033*unit.kilojoules_per_mole/(unit.nanometer**2))
        yu.addExclusion(i, i+1)
        ah.addExclusion(i, i+1)

    yu.setForceGroup(0)
    ah.setForceGroup(1)
    yu.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    ah.setNonbondedMethod(openmm.openmm.CustomNonbondedForce.CutoffPeriodic)
    hb.setUsesPeriodicBoundaryConditions(True)
    yu.setCutoffDistance(4*unit.nanometer)
    ah.setCutoffDistance(2*unit.nanometer)

    system.addForce(hb)
    system.addForce(yu)
    system.addForce(ah)

    integrator = openmm.openmm.LangevinIntegrator(temp*unit.kelvin,0.01/unit.picosecond,0.010*unit.picosecond) #10 fs timestep

    platform = openmm.Platform.getPlatformByName('CUDA')

    simulation = app.simulation.Simulation(pdb.topology, system, integrator, platform, dict(CudaPrecision='mixed'))

    check_point = '{:s}/restart.chk'.format(name)

    if os.path.isfile(check_point):
        print('Reading check point file')
        simulation.loadCheckpoint(check_point)
        simulation.reporters.append(app.dcdreporter.DCDReporter('{:s}/pretraj.dcd'.format(name),int(stride),append=True))
    else:
        simulation.context.setPositions(pdb.positions)
        simulation.minimizeEnergy()
        simulation.reporters.append(app.dcdreporter.DCDReporter('{:s}/pretraj.dcd'.format(name),int(stride)))

    simulation.reporters.append(app.statedatareporter.StateDataReporter('{:s}/traj.log'.format(name),int(stride),
             potentialEnergy=True,temperature=True,step=True,speed=True,elapsedTime=True,separator='\t'))

    simulation.step(nsteps)

    simulation.saveCheckpoint(check_point)

    genXTC(name,eqsteps)

In [ ]:
#@title <b><font color='#A79AB2'>Sequence analysis Toolbox</font></b>
def calc_seq_prop(seq,residues,Nc,Cc,Hc):
    """df: DataFrame to be populated with sequence properties
    r: DataFrame of aa-specific parameters"""
    model_nu = load('svr_model_nu.joblib')
    model_spr = load('svr_model_SPR.joblib')

    seq = list(seq).copy()
    fasta_kappa = np.array(seq.copy())
    N = len(seq)
    r = residues.copy()

    # calculate properties that do not depend on charges
    fK = sum([seq.count(a) for a in ['K']])/N
    fR = sum([seq.count(a) for a in ['R']])/N
    fE = sum([seq.count(a) for a in ['E']])/N
    fD = sum([seq.count(a) for a in ['D']])/N
    faro = sum([seq.count(a) for a in ['W','Y','F']])/N
    mean_lambda = np.mean(r.loc[seq].lambdas)

    pairs = np.array(list(itertools.combinations(seq,2)))
    pairs_indices = np.array(list(itertools.combinations(range(N),2)))
    # calculate sequence separations
    ij_dist = np.diff(pairs_indices,axis=1).flatten().astype(float)
    # calculate lambda sums
    ll = r.lambdas.loc[pairs[:,0]].values+r.lambdas.loc[pairs[:,1]].values
    # calculate SHD
    beta = -1
    shd = np.sum(ll*np.power(np.abs(ij_dist),beta))/N

    # fix charges
    if Nc == 1:
        r.loc['X'] = r.loc[seq[0]]
        r.loc['X','q'] = r.loc[seq[0],'q'] + 1.
        seq[0] = 'X'
        if r.loc['X','q'] > 0:
            fasta_kappa[0] = 'K'
        else:
            fasta_kappa[0] = 'A'
    if Cc == 1:
        r.loc['Z'] = r.loc[seq[-1]]
        r.loc['Z','q'] = r.loc[seq[-1],'q'] - 1.
        seq[-1] = 'Z'
        if r.loc['Z','q'] < 0:
            fasta_kappa[-1] = 'D'
        else:
            fasta_kappa[-1] = 'A'
    if Hc < 0.5:
        r.loc['H', 'q'] = 0
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'A'
    elif Hc >= 0.5:
        r.loc['H', 'q'] = 1
        fasta_kappa[np.where(np.array(seq) == 'H')[0]] = 'K'

    # calculate properties that depend on charges
    pairs = np.array(list(itertools.combinations(seq,2)))
    # calculate charge products
    qq = r.q.loc[pairs[:,0]].values*r.q.loc[pairs[:,1]].values
    # calculate SCD
    scd = np.sum(qq*np.sqrt(ij_dist))/N
    SeqOb = SequenceParameters(''.join(fasta_kappa))
    kappa = SeqOb.get_kappa()
    fcr = r.q.loc[seq].abs().mean()
    ncpr = r.q.loc[seq].mean()

    nu_svr = model_nu.predict([[scd,shd,kappa,fcr,mean_lambda]])[0]
    spr_svr = model_spr.predict([[scd,shd,mean_lambda]])[0]

    return np.around([fK, fR, fE, fD, faro, mean_lambda, shd, scd,
                      kappa, fcr, ncpr, nu_svr, spr_svr],3)

In [ ]:
#@title <b><font color='#A79AB2'>Simulation analysis Toolbox</font></b>
def autoblock(cv, multi=1, plot=False):
    block = BlockAnalysis(cv, multi=multi)
    block.SEM()

    if plot == True:
        plt.errorbar(block.stat[...,0], block.stat[...,1], block.stat[...,2], fmt='', color='k', ecolor='0.5')
        plt.scatter(block.bs, block.sem,zorder=10,c='tab:red')
        plt.xlabel('Block size')
        plt.ylabel('SEM')
        plt.show()

    return block.av, block.sem, block.bs

def fix_topology_pulchra(t,seq):
    cgtop = md.Topology()
    cgchain = cgtop.add_chain()
    for res in seq:
        cgres = cgtop.add_residue(res, cgchain)
        cgtop.add_atom('CA', element=md.element.carbon, residue=cgres)
    traj = md.Trajectory(t.xyz, cgtop, t.time, t.unitcell_lengths, t.unitcell_angles)
    traj = traj.superpose(traj, frame=0)
    return traj

def backmapping(name, traj):
    for i in range(traj.n_frames):
        traj[i].save_pdb('frame.pdb')
        subprocess.run(['./pulchra', 'frame.pdb'])
        if i == 0:
            traj_AA = md.load_pdb('frame.rebuilt.pdb')
        else:
            traj_AA += md.load_pdb('frame.rebuilt.pdb')
    top = md.Topology()
    chain = top.add_chain()
    for residue in traj_AA.top.residues:
        res = top.add_residue(residue.name, chain, resSeq=residue.index+1)
        for atom in residue.atoms:
            top.add_atom(atom.name, element=atom.element, residue=res)
    traj_AA = md.Trajectory(traj_AA.xyz, top, traj.time, traj.unitcell_lengths, traj.unitcell_angles)
    traj_AA[0].save_pdb(f'{name:s}/top_AA.pdb')
    traj_AA.save_dcd(f'{name:s}/traj_AA.dcd')
    return traj_AA

def plot_dist(ax,x,p,av,color='k'):
    ax.plot(x,p,c=color)
    ax.axvline(av,color=color)
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0,np.max(p)*1.1)

def plot_rew_dist(ax,x,p,av):
    ax.plot(x,p,c='tab:red')
    ax.axvline(av,0,100, color='tab:red')
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(0,np.max(p)*1.1)

def error_ratio(v1,v2,e1,e2):
    ratio = v1/v2
    return ratio*np.sqrt((e1/v1)**2+(e2/v2)**2)

def calc_shape_and_entropy(t,forces,residues,seq,temp):
    fasta = list(seq)
    masses = residues.loc[fasta,'MW'].values[np.newaxis,:,np.newaxis]

    # calculate conformational entropy per residue (DOI: 10.1021/ct500684w)
    prefactor = constants.Boltzmann*temp/constants.hbar**2
    forces = forces/np.sqrt(masses/1e3/constants.Avogadro)
    forces = forces.reshape(t.n_frames,-1)*1e3/constants.Avogadro*1e9
    sigma = LedoitWolf().fit(forces).covariance_
    eigenvalues = linalg.eigvalsh(sigma)
    kT_over_hbar_omega = constants.Boltzmann*temp*np.sqrt(prefactor/eigenvalues)
    SPR = np.sum(np.log(kT_over_hbar_omega+1))/len(seq) # R

    # calculate the center of mass
    cm = np.sum(t.xyz*masses,axis=1)/masses.sum()
    # calculate residue-cm distances
    si = t.xyz - cm[:,np.newaxis,:]
    q = np.einsum('jim,jin->jmn', si*masses,si)/masses.sum()
    trace_q = np.trace(q,axis1=1,axis2=2)
    # calculate rg
    rgarray = np.sqrt(trace_q)
    # calculate traceless matrix
    mean_trace = np.trace(q,axis1=1,axis2=2)/3
    q_hat = q - mean_trace.reshape(-1,1,1)*np.identity(3).reshape(-1,3,3)
    # calculate asphericity
    Delta_array = 3/2*np.trace(q_hat**2,axis1=1,axis2=2)/(trace_q**2)
    # calculate oblateness
    S_array = 27*linalg.det(q_hat)/(trace_q**3)
    # calculate ensemble averages
    block_tr_q_hat_2 = BlockAnalysis(np.trace(q_hat**2,axis1=1,axis2=2), multi=1)
    block_tr_q_hat_2.SEM()
    block_tr_q_2 = BlockAnalysis(trace_q**2, multi=1)
    block_tr_q_2.SEM()
    block_det_q_hat = BlockAnalysis(linalg.det(q_hat), multi=1)
    block_det_q_hat.SEM()
    block_tr_q_3 = BlockAnalysis(trace_q**3, multi=1)
    block_tr_q_3.SEM()
    Delta = 3/2*block_tr_q_hat_2.av/block_tr_q_2.av
    S = 27*block_det_q_hat.av/block_tr_q_3.av
    Delta_err = 3/2*error_ratio(block_tr_q_hat_2.av,block_tr_q_2.av,block_tr_q_hat_2.sem,block_tr_q_2.sem)
    S_err = 27*error_ratio(block_det_q_hat.av,block_tr_q_3.av,block_det_q_hat.sem,block_tr_q_3.sem)
    return rgarray, Delta_array, S_array, Delta, S, Delta_err, S_err, SPR

def Rij(traj,w=None):
    pairs = traj.top.select_pairs('all','all')
    d = md.compute_distances(traj,pairs)
    dmax = np.max(d)
    nres = traj.n_atoms
    ij = np.arange(2,nres,1)
    diff = [x[1]-x[0] for x in pairs]
    dij = np.empty(0)
    for i in ij:
        dij = np.append(dij,np.sqrt(np.average(np.mean(d[:,diff==i]**2,axis=1),weights=w,axis=0)))
    f = lambda x,R0,v : R0*np.power(x,v)
    popt, pcov = curve_fit(f,ij[ij>5],dij[ij>5],p0=[.4,.5])
    nu = popt[1]
    nu_err = pcov[1,1]**0.5
    R0 = popt[0]
    R0_err = pcov[0,0]**0.5
    return ij,dij,dmax,nu,nu_err,R0,R0_err

#energy functions
HALR = lambda r,s,l : 4*0.8368*l*((s/r)**12-(s/r)**6)
HASR = lambda r,s,l : 4*0.8368*((s/r)**12-(s/r)**6)+0.8368*(1-l)
HA = lambda r,s,l : np.where(r<2**(1/6)*s, HASR(r,s,l), HALR(r,s,l))
HASP = lambda r,s,l,rc : np.where(r<rc, HA(r,s,l)-HA(rc,s,l), 0)

#force functions
LJ_F = lambda r,s,rvec : -6*4*0.8368*(2*s**12/r**14-s**6/r**8)*rvec
HA_F = lambda r,s,l,rvec : np.where(r<2**(1/6)*s, LJ_F(r,s,rvec), l*LJ_F(r,s,rvec))
HASP_F = lambda r,s,l,rvec,rc : np.where(r<rc, HA_F(r,s,l,rvec), 0)

DH_F = lambda r,yukawa_eps,yukawa_kappa,rvec : -yukawa_eps*np.exp(-r*yukawa_kappa)*(1+r*yukawa_kappa)/r**3*rvec
DHSP_F = lambda r,yukawa_eps,yukawa_kappa,rvec,rc : np.where(r<rc, DH_F(r,yukawa_eps,yukawa_kappa,rvec), 0)

def calc_energy_map(t,df,seq,rc,temp,ionic):
    indices = t.top.select_pairs('all','all')
    mask = np.abs(indices[:,0]-indices[:,1])>1 #exclude bonded pairs
    indices = indices[mask]
    dvec = md.compute_displacements(t,indices) #vector distances between pairs for each frame
    d = linalg.norm(dvec,axis=2)
    pairs = np.array(list(itertools.combinations(list(seq),2)))
    pairs = pairs[mask]
    sigmas = 0.5*(df.loc[pairs[:,0]].sigmas.values+df.loc[pairs[:,1]].sigmas.values)
    lambdas = 0.5*(df.loc[pairs[:,0]].lambdas.values+df.loc[pairs[:,1]].lambdas.values)
    RT = 8.3145*temp*1e-3
    fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
    epsw = fepsw(temp)
    lB = 1.6021766**2/(4*np.pi*8.854188*epsw)*6.022*1000/RT
    # Calculate the inverse of the Debye length
    yukawa_kappa = np.sqrt(8*np.pi*lB*ionic*6.022/10)
    qq = df.loc[pairs[:,0]].q.values*df.loc[pairs[:,1]].q.values
    yukawa_eps = qq*lB*RT
    emap = np.zeros(pairs.shape[0])
    forces = np.zeros((t.n_frames,t.n_atoms,3))
    dstd = np.std(d,axis=0)
    for j,(r,rvec) in enumerate(zip(np.split(d,20,axis=0),np.split(dvec,20,axis=0))):
        emap += np.nansum(HASP(r,sigmas[np.newaxis,:],lambdas[np.newaxis,:],rc),axis=0)
        for i in range(t.n_atoms):
            ndx_pairs = np.any(indices==i,axis=1)
            forces[j*r.shape[0]:(j+1)*r.shape[0],i] = np.nansum(
                HASP_F(r[:,ndx_pairs,np.newaxis],sigmas[np.newaxis,ndx_pairs,np.newaxis],
                            lambdas[np.newaxis,ndx_pairs,np.newaxis],rvec[:,ndx_pairs],rc),axis=1)
            forces[j*r.shape[0]:(j+1)*r.shape[0],i] += np.nansum(DHSP_F(r[:,ndx_pairs,np.newaxis],
                            yukawa_eps[np.newaxis,ndx_pairs,np.newaxis],yukawa_kappa,rvec[:,ndx_pairs],4),axis=1)
    return indices, emap/d.shape[0], forces, dstd

def Ree(t):
    return md.compute_distances( t, atom_pairs=np.array([[ 0,  len(list(t.top.atoms))-1]]) )[...,0]

def maps(traj,residues,seq,temp,ionic):
    #energy maps
    df_emap = pd.DataFrame(index=range(traj.n_atoms),columns=range(traj.n_atoms),dtype=float)
    df_dstd = pd.DataFrame(index=range(traj.n_atoms),columns=range(traj.n_atoms),dtype=float)
    pairs, emap, forces, dstd = calc_energy_map(traj,residues,seq,2.0,temp,ionic)
    for k,(i,j) in enumerate(pairs):
        df_emap.loc[i,j] = emap[k]
        df_emap.loc[j,i] = emap[k]
        df_dstd.loc[i,j] = dstd[k]
        df_dstd.loc[j,i] = dstd[k]
    return df_emap, forces, df_dstd

def kde(a, w=None, phi_eff=None, min_=None, max_=None):
    if type(w) == 'NoneType':
        w = np.full(len(a), 1)
    if min_ == None:
        min_ = np.min(a)
    if max_ == None:
        max_ = np.max(a)
    x = np.linspace( min_, max_, num = 50 )
    d = scs.gaussian_kde( a, bw_method = "silverman", weights = w ).evaluate(x)
    u = DescrStatsW( a, weights = w )
    n_eff = phi_eff*a.size if phi_eff!=None else a.size
    return x,d/np.sum(d),u.mean,u.std/np.sqrt(n_eff)

In [ ]:
#@title <b><font color='#FA003F'>BME Toolbox</font></b>
import BME as BME

def iBME(calc_file,exp_file,THETAS=np.array([1,10,20,50,75,100,200,400,750,1000,5000,10000])):
    W = []
    STATS = []

    for t in THETAS:
        print('Reweighting with theta={}'.format(t))
        rew = BME.Reweight('ibme_t{}'.format(t))
        rew.load(exp_file,calc_file)
        rew.ibme(theta=t, iterations=25, ftol=0.001)

        W.append(rew.get_ibme_weights()[-1])
        STATS.append(rew.get_ibme_stats()[-1])
        print('chi2={:.2f}, phi_eff={:.2f}'.format(STATS[-1][1],STATS[-1][2]))

    return THETAS, np.array(STATS), np.array(W)

def theta_loc(thetas, stats):
    kneedle = KneeLocator(stats[...,2], stats[...,1], S=1, curve="convex", direction="increasing")
    choice = np.array(thetas)[stats[...,2]==kneedle.knee][0]
    return choice

In [ ]:
%%time
#@title <b><font color='#45B69C'>1 - Run MD simulation</font></b>
#@markdown Simulation time (ns):
Simulation_time = "AUTO" #@param {type:"raw"}

N_res = len(SEQUENCE)
N_save = 7000 if N_res < 150 else int(np.ceil(3e-4*N_res**2)*1000)

if Simulation_time == "AUTO":
    nsteps = 1010*N_save
    print('AUTO simulation length selected. Running for {} ns'.format(nsteps*0.01/1000))
else:
    nsteps = float(Simulation_time)*1000/0.01//N_save*N_save
try:
    shutil.rmtree(NAME)
except:
    pass
simulate(residues,NAME,list(SEQUENCE),temp=Temperature,ionic=Ionic_strength,Nc=Nc,Cc=Cc,
         Hc=Hc,nsteps=nsteps,stride=N_save,eqsteps=10)

In [ ]:
#@title <b><font color='#45B69C'>2 - Simulation analysis</font></b>
if not os.path.isfile(f'{NAME:s}/traj.xtc'):
    url = 'https://github.com/gitesei/EnsembleLab/blob/main/'
    wget.download(url+f'{NAME:s}/traj.xtc?raw=true')
    wget.download(url+f'{NAME:s}/top.pdb?raw=true')
    if not os.path.isdir(f'{NAME:s}'):
        os.mkdir(f'{NAME:s}')
    shutil.move('traj.xtc',f'{NAME:s}/traj.xtc')
    shutil.move('top.pdb',f'{NAME:s}/top.pdb')
    traj = md.load_xtc('{:s}/traj.xtc'.format(NAME), top='{:s}/top.pdb'.format(NAME))[:1000]
else:
    traj = md.load_xtc('{:s}/traj.xtc'.format(NAME), top='{:s}/top.pdb'.format(NAME))

df_emap, forces, df_dstd = maps(traj,residues,SEQUENCE,Temperature,Ionic_strength)

rg_array, D_array, S_array, D, S, D_err, S_err, SPR = calc_shape_and_entropy(traj,forces,
                                                                    residues,SEQUENCE,Temperature)

rg, rg_err, rg_blocksize = autoblock(rg_array)
x_rg, p_rg, _, _ = kde(rg_array)
x_D, p_D, _, _ = kde(D_array)
x_S, p_S, _, _ = kde(S_array)

ree_array = Ree(traj)
ree, ree_err, ree_blocksize = autoblock(ree_array)
x_ree, p_ree, _, _ = kde(ree_array)

ij,dij,dmax,nu,nu_err,R0,R0_err = Rij(traj)

# Plot results
mpl.rcParams.update({'font.size': 10})
fig, axs = plt.subplots(2, 3, figsize=(7,3.5), facecolor='w', dpi=300, layout='tight')
axs = axs.flatten()

axs[0].plot(x_rg, p_rg)
top = p_rg.max()+0.1*p_rg.max()
axs[0].axvline(rg)
axs[0].axvspan(exp_data[NAME]['exp_rg']-exp_data[NAME]['exp_rg_err'],
               exp_data[NAME]['exp_rg']+exp_data[NAME]['exp_rg_err'], lw=0,
               color='k',
               alpha=.5)
axs[0].set_xlabel(r'$R_g$ (nm)')
axs[0].set_ylabel(r'$p(R_g)$')
axs[0].set_ylim(0,top)
axs[0].fill_between([rg-rg_err,rg+rg_err],0,top,alpha=0.3)

axs[1].plot(x_D, p_D)
top = p_D.max()+0.1*p_D.max()
axs[1].axvline(D)
axs[1].set_xlabel(r'$\Delta$')
axs[1].set_ylabel(r'$p(\Delta)$')
axs[1].set_ylim(0,top)
axs[1].fill_between([D-D_err,D+D_err],0,top,alpha=0.3)

axs[2].plot(x_S, p_S)
top = p_S.max()+0.1*p_S.max()
axs[2].axvline(S)
axs[2].set_xlabel(r'$S$')
axs[2].set_ylabel(r'$p(S)$')
axs[2].set_ylim(0,top)
axs[2].fill_between([S-S_err,S+S_err],0,top,alpha=0.3)

axs[3].plot(x_ree, p_ree)
top = p_ree.max()+0.1*p_ree.max()
axs[3].vlines(ree,0,top)
axs[3].set_xlabel(r'$R_{ee}$ (nm)')
axs[3].set_ylabel(r'$p(R_{ee})$')
axs[3].set_ylim(0,top)

axs[4].plot(ij,dij)
dij_fit = R0*np.power(ij,nu)
axs[4].plot(ij, dij_fit,c='0.3',ls='dashed',label='Fit')
axs[4].set_xlabel('$|i-j|$')
axs[4].set_ylabel(r'$\sqrt{\langle R_{ij}^2 \rangle}$ (nm)')
axs[4].text(0.05, 0.9, r'$\nu$={:.2f}'.format(nu), horizontalalignment='left',
            verticalalignment='center', transform=axs[4].transAxes, fontsize=10)
axs[4].legend(loc='lower right')

im = axs[5].imshow(df_emap*1e3,extent=[1, df_emap.shape[0], 1, df_emap.shape[0]],origin='lower',
                   aspect='equal',vmin=-9,vmax=0,cmap=plt.cm.Blues_r)
cb = plt.colorbar(im, ax=axs[5], label=r'$U$ (J mol$^{-1}$)',fraction=0.05, pad=0.04)
cb.set_ticks([0,-3,-6,-9])
axs[5].set_xlabel('Residue #')
axs[5].set_ylabel('Residue #')

plt.savefig(f'{NAME}/conformational_properties.pdf', dpi=300, facecolor='w', edgecolor='w', orientation='portrait',
            bbox_inches='tight')
plt.show()

df_means = pd.DataFrame(data=np.c_[[exp_data[NAME]['exp_rg'],exp_data[NAME]['exp_rg_err']],
                                   [rg,rg_err],[ree,ree_err],[nu,nu_err],[D,D_err],[S,S_err],[SPR,0]],
                        columns=['Exp <Rg> (nm)','<Rg> (nm)','<Ree> (nm)','nu','<Delta>','<S>','S_conf/N (kB)'],
                        index=['Value','Error'])

df_means.to_csv(f'{NAME}/conf_properties.csv')
df_emap.to_csv(f'{NAME}/energy_map.csv')

df_means

In [ ]:
#@title <b><font color='#F26419'>2.1 - Sequence analysis</font></b>
df_seq = pd.DataFrame(columns=['fK','fR','fE','fD','fARO','mean_lambda','SHD','SCD','kappa',
                           'FCR','NCPR','nu_SVR','S_conf,SVR/N (kB)'])
f = open('seq.fasta', 'r').readlines()
NAME = f[0][1:].strip()
SEQUENCE = f[1].strip()
Temperature, Ionic_strength, Hc, Nc, Cc = np.loadtxt('env_settings.txt', unpack=True)

df_seq.loc[NAME] = calc_seq_prop(SEQUENCE,residues,Nc,Cc,Hc)

df_seq.to_csv(f'{NAME}/sequence_properties.csv')

df_seq

In [ ]:
%%time
#@title <b><font color='#ffc413'>2.2 - Generate all-atom trajectory</font></b>
THREE_LETTER_SEQ = [residues.three[x] for x in SEQUENCE]
t_cg = fix_topology_pulchra(traj, THREE_LETTER_SEQ)

#downsample the trajectory based on the size of blocks of correlated Rg and Ree values
#n_skip = int((rg_blocksize+ree_blocksize)/2)

#t_cg = t_cg[::n_skip]

backmapping(NAME, t_cg)

In [ ]:
#@title <b><font color='#058ED9'>3 - Ensemble reweighting with experimental data</b>: getting data ready</font>
#correct experimental errors with BIFT
f = open('inputfile.dat','w')
f.write("saxs_input.dat\n\n\n\n{}\n\n\n\n\n\n\n\n\n\n\n\n".format(dmax))
f.close()
subprocess.run('./bift < inputfile.dat'.split())
np.savetxt('{:s}_bift.dat'.format(NAME), np.loadtxt('rescale.dat'), header=' DATA=SAXS')
print('Experimental errors on SAXS intensities have been corrected with BIFT')
print('Factor used for rescaling errors is: {}'.format(np.loadtxt('scale_factor.dat')[0,1]))
print('SAXS data with corrected errors is in {:s}_bift.dat\n'.format(NAME))

#SAXS
print('Calculating SAXS from all-atom trajectory...')
t_aa = md.load_dcd('{}/traj_AA.dcd'.format(NAME), top='{}/top_AA.pdb'.format(NAME))
for i,f in enumerate(t_aa):
    f.save_pdb('frame.pdb')
    pepsi_comm = './Pepsi-SAXS frame.pdb {:s}_bift.dat -o saxs.dat -cst --cstFactor 0 --I0 1.0 --dro 1.0 --r0_min_factor 1.025 --r0_max_factor 1.025 --r0_N 1'.format(NAME)
    subprocess.run(pepsi_comm.split())
    if i == 0:
        calc_saxs = np.loadtxt('saxs.dat')[...,3]
    else:
        calc_saxs = np.vstack((calc_saxs,np.loadtxt('saxs.dat')[...,3]))
col0 = np.arange(0,len(calc_saxs)).reshape(len(calc_saxs),1)
calc_saxs = np.hstack((col0,calc_saxs))
np.savetxt('calc_saxs.dat', calc_saxs)

In [ ]:
#@title <b><font color='#058ED9'>3.1 - Execute reweighting</b></font>
def f(theta_index):
    mpl.rcParams.update({'font.size': 10})
    fig = plt.figure(layout='constrained',dpi=300,figsize=(7,3))
    plt.scatter(stats[...,2],stats[...,1], c='k')
    plt.scatter(stats[...,2][theta_index],stats[...,1][theta_index], c='tab:red',label=r'Chosen $\theta$')
    plt.xlabel(r'$\phi_{eff}$',fontsize=10)
    plt.ylabel(r'$\chi^2_r$',fontsize=10)
    plt.legend()
    plt.show()
    return theta_index

thetas, stats, weights = iBME('calc_saxs.dat', '{:s}_bift.dat'.format(NAME))

THETA_LOCATOR = "AUTO" #@param ["AUTO", "INTERACTIVE"]
if THETA_LOCATOR == "AUTO":
    choice = theta_loc(thetas, stats)
    mpl.rcParams.update({'font.size': 10})
    fig = plt.figure(layout='constrained',dpi=300,figsize=(7,3))
    plt.scatter(stats[...,2],stats[...,1], c='k')
    ndx = np.where(thetas==choice)[0][0]
    plt.scatter(stats[...,2][ndx],stats[...,1][ndx], c='tab:red',label=r'Chosen $\theta$')
    plt.xlabel(r'$\phi_{eff}$',fontsize=10)
    plt.ylabel(r'$\chi^2_r$',fontsize=10)
    plt.legend()
    plt.show()
elif THETA_LOCATOR == "INTERACTIVE":
    interactive_plot = interactive(f, theta_index=(0,len(thetas)-1))
    display(interactive_plot)

In [ ]:
#@title <b><font color='#058ED9'>3.2 Analyze reweighted ensemble</b></font>
if THETA_LOCATOR == "INTERACTIVE":
    ndx = interactive_plot.result
    choice = thetas[ndx]

print('Reweighting using theta={}'.format(choice))

q, I_exp, err = np.loadtxt('{}_bift.dat'.format(NAME), unpack=True)
I_prior = np.average(np.loadtxt('calc_saxs.dat')[...,1:],axis=0)
I_post = np.average(np.loadtxt(list(filter(lambda x: x.startswith('ibme_t{}_'.format(choice)) and x.endswith('.calc.dat'), os.listdir('.')))[0])[...,1:], axis=0, weights=weights[ndx])
wlr = 1/(err**2)
model = LinearRegression()
model.fit(I_prior.reshape(-1,1),I_exp,wlr)
a = model.coef_[0]
b = model.intercept_
I_prior = a*I_prior+b

mpl.rcParams.update({'font.size': 10})
fig, axs = plt.subplots(3, 2, figsize=(7,6), facecolor='w', dpi=300, layout='constrained')
axs = axs.flatten()

axs[0].errorbar(q,I_exp,err,c='tab:gray',alpha=.5)
axs[0].plot(q,I_prior, zorder=500)
axs[0].plot(q,I_post, color='tab:red', zorder=1000)
axs[0].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[0].set_ylabel('Intensity')
axs[0].set_ylim(0,I_post.max()*1.1)
axs[0].set_xlim(0,4)

axs[1].errorbar(q,I_exp,err,lw=1,c='tab:gray',alpha=.5)
axs[1].plot(q,I_prior, zorder=500)
axs[1].plot(q,I_post, color='tab:red', zorder=1000)
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[1].set_ylabel('Intensity')
axs[1].set_ylim(0,I_prior.max()*1.1)

kratky_exp = (q**2)*I_exp
kratky_err = (q**2)*err
axs[2].errorbar(q,kratky_exp,kratky_err,c='tab:gray',alpha=.5)
axs[2].plot(q,(q**2)*I_prior, zorder=500)
axs[2].plot(q,(q**2)*I_post, color='tab:red', zorder=1000)
axs[2].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[2].set_ylabel(r'$q^2I$')
axs[2].set_ylim(0,((q**2)*I_post).max()*1.1)
axs[2].set_xlim(0,4)

axs[3].plot(q, (I_exp-I_prior)/err, color='tab:blue', lw=1)
axs[3].plot(q, (I_exp-I_post)/err, color='tab:red', lw=1)
axs[3].set_xlabel(r'$q$ (nm$^{-1}$)')
axs[3].set_ylabel(r'$(I^\mathrm{EXP}-I^\mathrm{CALC})/\sigma$')

x_rg_rew, p_rg_rew, rg_av_rew, rg_se_rew = kde(rg_array, w=weights[ndx], phi_eff=stats[...,2][ndx], min_=np.min(rg_array), max_=np.max(rg_array))
x_ree_rew, p_ree_rew, ree_av_rew, ree_se_rew = kde(ree_array, w=weights[ndx], phi_eff=stats[...,2][ndx], min_=np.min(ree_array), max_=np.max(ree_array))
ij, dij_rew, dmax_rew, nu_rew, nu_err_rew, _, _ = Rij(traj, w=weights[ndx])

plot_dist(axs[4], x_rg, p_rg, rg,color='tab:blue')
plot_rew_dist(axs[4], x_rg_rew, p_rg_rew, rg_av_rew)
axs[4].axvspan(exp_data[NAME]['exp_rg']-exp_data[NAME]['exp_rg_err'],
               exp_data[NAME]['exp_rg']+exp_data[NAME]['exp_rg_err'], lw=0,
               color='k',
               alpha=.5)
axs[4].set_xlabel(r'$R_g$ (nm)')
axs[4].set_ylabel(r'$p(R_g)$')
axs[4].annotate(xy=(0.95,0.85), xycoords='axes fraction', text=r'$\langle R_g \rangle={:.2f}$ nm'.format(rg_av_rew), color='tab:red', fontsize=10, horizontalalignment='right')

axs[5].plot(ij,dij,c='tab:blue')
axs[5].plot(ij,dij_rew,nu_rew,c='tab:red')
axs[5].set_xlabel('$|i-j|$')
axs[5].set_ylabel(r'$\langle R_{ij} \rangle$ (nm)')
axs[5].annotate(xy=(0.05,0.85), xycoords='axes fraction', text=r'$\nu$={:.2f}'.format(nu_rew), color='tab:red', fontsize=10)

try:
    os.mkdir(f'{NAME}/REWEIGHTED')
except:
    pass

plt.savefig(f'{NAME}/REWEIGHTED/SAXS_reweighting.pdf', dpi=300, facecolor='w', edgecolor='w', orientation='portrait',
            bbox_inches='tight')
plt.show()

In [ ]:
#@title <b><font color='#058ED9'>3.3 - Reweighted Conformational Properties</b></font>
df_reweighted = pd.DataFrame(data=np.c_[[exp_data[NAME]['exp_rg'],exp_data[NAME]['exp_rg_err']],
                                   [rg_av_rew,rg_se_rew],
                                   [ree_av_rew,ree_se_rew],
                                   [nu_rew,nu_err_rew]],
                        columns=['Exp <Rg> (nm)','<Rg> (nm)','<Ree> (nm)','nu'],
                        index=['Value','Error'])

df_reweighted

In [ ]:
#@title <b><font color='#058ED9'>3.4 - Conformational Properties Before Reweighting</b></font>
pd.DataFrame(data=np.c_[[exp_data[NAME]['exp_rg'],exp_data[NAME]['exp_rg_err']],
                                   [rg,rg_err],[ree,ree_err],[nu,nu_err]],
                        columns=['Exp <Rg> (nm)','<Rg> (nm)','<Ree> (nm)','nu'],
                        index=['Value','Error'])

In [ ]:
#@title <b><font color='#72A276'>4 - Download results</b></font>
try:
    os.remove(f'{NAME}/pretraj.dcd')
    os.remove(f'{NAME}/restart.chk')
except:
    pass
try:
    shutil.move('seq.fasta',f'{NAME}/seq.fasta','w')
except:
    pass
try:
    shutil.move('env_settings.txt',f'{NAME}/env_settings.txt','w')
except:
    pass
try:
    os.mkdir(f'{NAME}_EnsembleLab')
except:
    pass
try:
    shutil.copytree(f'{NAME}/REWEIGHTED',f'{NAME}_EnsembleLab/REWEIGHTED')
    shutil.rmtree(f'{NAME}/REWEIGHTED')
except:
    pass
try:
    shutil.copytree(f'{NAME}',f'{NAME}_EnsembleLab/SIMULATION')
except:
    pass

pd.DataFrame(data=np.c_[rg_array,ree_array,D_array,S_array,weights[ndx]],
             columns=['Rg (nm)','Ree (nm)','Delta','S','weights']).to_csv(
             f'{NAME}/time_series_Rg_Ree_Delta_S.csv'.format(NAME))
pd.DataFrame(data=np.c_[ij,dij],
             columns=['ij','Rij (nm)']).to_csv(f'{NAME}/scaling_profile.csv')
pd.DataFrame(data=np.c_[ij,dij_rew],
             columns=['ij','Reweighted Rij (nm)']).to_csv(f'{NAME}_EnsembleLab/REWEIGHTED/reweighted_scaling_profile.csv')
df_reweighted = pd.DataFrame(data=np.c_[[exp_data[NAME]['exp_rg'],exp_data[NAME]['exp_rg_err']],
                                   [rg_av_rew,rg_err],
                                   [ree_av_rew,ree_err],
                                   [nu_rew,nu_err_rew]],
                        columns=['Exp <Rg> (nm)','<Rg> (nm)','<Ree> (nm)','nu'],
                        index=['Value','Error'])
df_means.to_csv(f'{NAME}_EnsembleLab/REWEIGHTED/reweighted_conf_properties.csv')

np.savetxt(f'{NAME}_EnsembleLab/REWEIGHTED/saxs.dat', np.vstack((q,I_exp,err,I_prior,I_post)).T, header='q I(exp) err(bift) I(prior) I(post)')
wget.download('https://raw.githubusercontent.com/gitesei/EnsembleLab/main/utils/readme_download_simulation.txt')
fout = open(f'{NAME}_EnsembleLab/README', 'w')
fout.write(''.join(open('readme_download_simulation.txt', 'r').readlines()))
fout.write('\n')
wget.download('https://raw.githubusercontent.com/gitesei/EnsembleLab/main/utils/readme_download_reweighted.txt')
fout.write(''.join(open('readme_download_reweighted.txt', 'r').readlines()))
fout.close()

zipper = f'zip -r {NAME}_EnsembleLab.zip {NAME}_EnsembleLab'

subprocess.run(zipper.split())
files.download(f'{NAME}_EnsembleLab.zip')

In [ ]:
#@title <b><font color='#ffc413'>5 - Calculate FRET efficiency
# Create a MDAnalysis.Universe object for the protein structure.
u = MDAnalysis.Universe(f'{NAME}/top_AA.pdb',
                        f'{NAME}/traj_AA.dcd')

alexa_D = exp_data[NAME]['alexa'][0].split(' ')[0]
alexa_A = exp_data[NAME]['alexa'][1].split(' ')[0]

# Create instance of the FRETpredict class
FRET = FRETpredict(protein=u, residues=exp_data[NAME]['sites'], chains=['A', 'A'], temperature=296,
                  donor=f'AlexaFluor {alexa_D:s}', acceptor=f'AlexaFluor {alexa_A:s}', electrostatic=True,
                  libname_1=f'AlexaFluor {exp_data[NAME]["alexa"][0]:s} cutoff20',
                  libname_2=f'AlexaFluor {exp_data[NAME]["alexa"][1]:s} cutoff20',
                  output_prefix=f'FRET/{NAME}')

# Run FRET efficiency calculations.
FRET.run()

site_D = exp_data[NAME]['sites'][0]
site_A = exp_data[NAME]['sites'][1]

sim_E = pd.read_pickle(f'FRET/{NAME:s}-data-{site_D:d}-{site_A:d}.pkl').loc['Edynamic1','Average']
sim_E_err = pd.read_pickle(f'FRET/{NAME:s}-data-{site_D:d}-{site_A:d}.pkl').loc['Edynamic1','SE']
distance,dye_dye_distr = np.loadtxt(f'FRET/{NAME:s}-{site_D:d}-{site_A:d}.dat',unpack=True)

round_E = abs(int(np.log10(sim_E_err)))+1

fret_data = pd.DataFrame(data=np.c_[[exp_data[NAME]['exp_E'],exp_data[NAME]['exp_E_err']],
                        [round(sim_E,round_E),round(sim_E_err,round_E)]],
                    columns=['Exp FRET <E>','Sim FRET <E>'],
                    index=['Value','Error'])

fret_data

In [ ]:
#@title <b><font color='#ffc413'>5.1 - Reweight FRET efficiency
FRET.reweight(reweight_prefix=f'FRET/{NAME}', user_weights=weights[ndx])

site_D = exp_data[NAME]['sites'][0]
site_A = exp_data[NAME]['sites'][1]

sim_E = pd.read_pickle(f'FRET/{NAME:s}-data-{site_D:d}-{site_A:d}.pkl').loc['Edynamic1','Average']
sim_E_err = pd.read_pickle(f'FRET/{NAME:s}-data-{site_D:d}-{site_A:d}.pkl').loc['Edynamic1','SE']
distance,dye_dye_distr_reweighted = np.loadtxt(f'FRET/{NAME:s}-{site_D:d}-{site_A:d}.dat',unpack=True)

round_E = abs(int(np.log10(sim_E_err)))+1

reweighted_fret_data = pd.DataFrame(data=np.c_[[exp_data[NAME]['exp_E'],exp_data[NAME]['exp_E_err']],
                        [round(sim_E,round_E),round(sim_E_err,round_E)]],
                    columns=['Exp FRET <E>','Reweighted Sim FRET <E>'],
                    index=['Value','Error'])

reweighted_fret_data

In [ ]:
#@title <b><font color='#ffc413'>5.2 - Comparison between dye-dye distributions
mpl.rcParams.update({'font.size': 10})
fig = plt.figure(figsize=(3.5,3.5), dpi=300)

plt.plot(distance[dye_dye_distr>0.01],
         dye_dye_distr[dye_dye_distr>0.01],label='CALVADOS')
plt.plot(distance[dye_dye_distr_reweighted>0.01],
         dye_dye_distr_reweighted[dye_dye_distr_reweighted>0.01],color='tab:red',label='CALVADOS+BME')
plt.ylabel('$P(r_{dye-dye})$')
plt.xlabel('$r_{dye-dye}$, Dye-dye Separation  /  nm')
plt.legend()
plt.show()